# Estimating Pandemic Risk

Here, I will compare different estimates for the following types of pandemics:
1. Natural pandemics (based on estimates from [Marani et al. 2019](https://doi.org/10.1073/pnas.2105482118))
2. Accidental pandemics (based on estimates from [Klotz 2021](https://armscontrolcenter.org/wp-content/uploads/2017/04/LWC-paper-final-version-for-CACNP-website.pdf))
3. Deliberate pandemics

In [1]:
from ipywidgets import widgets, HBox, VBox, HTML, Layout, interactive, interactive_output, Output
from IPython.display import clear_output
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.templates.default = 'plotly_white'

#################################
# Define parameters and functions
#################################

# Global variables
num_years = 100
population = 9.2e9
num_simulations = 10000
# Accidental
P_release = 0.00246
P_seeds_pandemic_min = 0.05
P_seeds_pandemic_max = 0.4
num_facilities = 14
fatality_rate = 0.025
infection_rate = 0.15
# Other
COLORS = {
    "accidental": "#1f77b4",  # blue
    "natural": "#ff7f0e",     # orange
    "deliberate": "#2ca02c"   # green
}

In /home/phil/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/phil/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/phil/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/phil/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/phil/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-d

In [2]:
# Source link
klotz_2021 = "https://armscontrolcenter.org/wp-content/uploads/2017/04/LWC-paper-final-version-for-CACNP-website.pdf"

# Define a consistent layout for the description widgets
desc_layout = widgets.Layout(width='600px')

# Create HTML widgets for descriptions with links
# Global variables
num_years_desc = HTML(value="Number of years", layout=desc_layout)
population_desc = HTML(value="World population", layout=desc_layout)
num_simulations_desc = HTML(value="Number of simulations", layout=desc_layout)
# Accidental release
P_release_desc = HTML(
    value=f"Probability of community release from a single facility (with one or many labs) in a single year (default value: estimated in <a href='{klotz_2021}'>Klotz 2021</a>):", 
    layout=desc_layout
)
P_seeds_pandemic_min_desc = HTML(
    value=f"Probability Virus Seeds Pandemic (Min) (default value: estimated in <a href='{klotz_2021}'>Klotz 2021</a>):", 
    layout=desc_layout
)
P_seeds_pandemic_max_desc = HTML(
    value=f"Probability Virus Seeds Pandemic (Max) (default value: estimated in <a href='{klotz_2021}'>Klotz 2021</a>):", 
    layout=desc_layout
)
num_facilities_desc = HTML(
    value=f"Number of Facilities (default value: the number of HPAI facilities as stated in <a href='{klotz_2021}'>Klotz 2021</a>):", 
    layout=desc_layout
)
fatality_rate_desc = HTML(
    value=f"Case fatality rate (default value: 1918 influenza CFR, <a href='{klotz_2021}'>see Klotz 2021</a>):", 
    layout=desc_layout
)
infection_rate_desc = HTML(
    value=f"Fraction of Population Infected (default value: % infected in typical flu season, <a href='{klotz_2021}'>see Klotz 2021</a>):", 
    layout=desc_layout
)

# Create text widgets without descriptions 
num_years_widget = widgets.IntText(value=num_years, layout=widgets.Layout(width='200px'))
population_widget = widgets.FloatText(value=population, layout=widgets.Layout(width='200px'))
num_simulations_widget = widgets.FloatLogSlider(value=num_simulations, base=10, min=2, max=6, step=1, layout=widgets.Layout(width='400px'))
P_release_widget = widgets.FloatText(value=P_release, layout=widgets.Layout(width='200px'))
P_seeds_pandemic_min_widget = widgets.FloatText(value=P_seeds_pandemic_min, layout=widgets.Layout(width='200px'))
P_seeds_pandemic_max_widget = widgets.FloatText(value=P_seeds_pandemic_max, layout=widgets.Layout(width='200px'))
num_facilities_widget = widgets.IntText(value=num_facilities, layout=widgets.Layout(width='200px'))
fatality_rate_widget = widgets.FloatText(value=fatality_rate, layout=widgets.Layout(width='200px'))
infection_rate_widget = widgets.FloatText(value=infection_rate, layout=widgets.Layout(width='200px'))

# Define the layout for the subtitles
subtitle_layout = Layout(padding='0px 0px 0px 0px', font_size='1.2em')
global_variables_subtitle = HTML(value="<strong>Global Variables</strong>")
accidental_release_subtitle = HTML(value="<strong>Accidental Release Variables</strong>")

# Group the widgets
global_variables_widgets = VBox([
    global_variables_subtitle,
    HBox([num_years_desc, num_years_widget]),
    HBox([population_desc, population_widget]),
    HBox([num_simulations_desc, num_simulations_widget]),
])

accidental_release_widgets = VBox([
    accidental_release_subtitle,
    HBox([P_release_desc, P_release_widget]),
    HBox([P_seeds_pandemic_min_desc, P_seeds_pandemic_min_widget]),
    HBox([P_seeds_pandemic_max_desc, P_seeds_pandemic_max_widget]),
    HBox([num_facilities_desc, num_facilities_widget]),
    HBox([fatality_rate_desc, fatality_rate_widget]),
    HBox([infection_rate_desc, infection_rate_widget]),
])

By default, I will estimate the total expected number of deaths from each type of pandemic over the next century. For simplicity, I will assume a fixed population of 9.2 billion people over the next century (based on [UN population projections](https://www.worldometers.info/world-population/world-population-projections/)):

$
\text{Average population} = \frac{\text{Starting population} + \text{Population after 100 years}}{2} = \frac{8.0 \text{ billion} + 10.3 \text{ billion}}{2} = 9.2 \text{ billion}
$

I will be using Monte Carlo simulations to estimate the expected number of deaths from each type of pandemic. All of these paramters, such as the time horizon, population size, and number of simulations, can be changed below.

In [3]:
display(global_variables_widgets)

## 1. Understanding Pandemic Risk from Accidental Release

We'll be examining the potential risk of a pandemic from the accidental release of viruses from research facilities. This analysis will involve three main sections:

1. Probability a Single Facility in a Single Year Seeds a Pandemic
2. Number of Pandemics Seeded by Any matHPAI Facility
3. Worldwide Fatalities from the Pandemic

Note, limitations of this analysis include:
- The results are highly dependent on the probabilities of release computed in [Klotz 2020](https://armscontrolcenter.org/wp-content/uploads/2020/03/Quantifying-the-risk-9-17-Supplementary-material-at-end.pdf)
- We are only considering the risk from highly pathogenic avian influenza (HPAI) viruses, and not other types of viruses
- We are only considering the risk from research facilities, and not other sources of accidental release
- We are assuming that the number of facilities conducting HPAI research will remain constant over the next century


In [4]:
display(accidental_release_widgets)

def compute_accidental(P_release, P_seeds_pandemic_min, P_seeds_pandemic_max, num_simulations, population, infection_rate, fatality_rate, num_facilities, num_years):
    num_simulations = int(num_simulations)
    
    # Compute P_seeds_pandemic_samples and P_single_pandemic_samples
    P_seeds_pandemic_samples = np.random.uniform(P_seeds_pandemic_min, P_seeds_pandemic_max, num_simulations)
    P_single_pandemic_samples = P_release * P_seeds_pandemic_samples

    # Compute total_pandemics_per_simulation
    total_pandemics_per_simulation = P_single_pandemic_samples * num_years * num_facilities
    
    # Compute fatalities_samples
    fatalities_samples = total_pandemics_per_simulation * population * infection_rate * fatality_rate
    
    return P_seeds_pandemic_samples, P_single_pandemic_samples, total_pandemics_per_simulation, fatalities_samples


# Create individual Output widgets for each plot
output_P_single_pandemic = Output()
output_P_any_facility = Output()
output_expected_fatalities = Output()

# Modify the plotting functions to render within these output widgets
def plot_P_single_pandemic(P_single_pandemic_samples):
    with output_P_single_pandemic:
        clear_output(wait=True)  # Clear the current content
        fig = px.histogram(
            P_single_pandemic_samples, 
            nbins=50, 
            labels={'value': 'P(single_pandemic)'}, 
            title="Distribution of P(single_pandemic)", 
            color_discrete_sequence=[COLORS["accidental"]],
            histnorm='probability'
        )
        fig.show()

def plot_P_any_facility(P_any_facility_samples):
    with output_P_any_facility:
        clear_output(wait=True)  # Clear the current content
        fig = px.histogram(
            P_any_facility_samples, 
            nbins=50, 
            labels={'value': 'Number of Pandemics'}, 
            title="Distribution of Number of Pandemics", 
            color_discrete_sequence=[COLORS["accidental"]],
            histnorm='probability'
        )
        fig.show()

def plot_expected_fatalities(fatalities_samples):
    with output_expected_fatalities:
        clear_output(wait=True)  # Clear the current content
        fig = px.histogram(
            fatalities_samples, 
            nbins=50, 
            labels={'value': 'Expected fatalities'}, 
            title="Distribution of Expected Fatalities", 
            color_discrete_sequence=[COLORS["accidental"]],
            histnorm='probability'
        )
        fig.show()

# Update the update_plots function
def update_plots(P_release, P_seeds_pandemic_min, P_seeds_pandemic_max, num_simulations, population, infection_rate, fatality_rate, num_facilities, num_years):
    # Compute the accidental risks
    P_seeds_pandemic_samples, P_single_pandemic_samples, pandemics_per_year_samples, fatalities_samples = compute_accidental(
        P_release, P_seeds_pandemic_min, P_seeds_pandemic_max, num_simulations, population, infection_rate, fatality_rate, num_facilities, num_years)
    
    # Plot the results
    plot_P_single_pandemic(P_single_pandemic_samples)
    plot_P_any_facility(pandemics_per_year_samples)
    plot_expected_fatalities(fatalities_samples)

output = interactive_output(
    update_plots, 
    {
        'P_release': P_release_widget,
        'P_seeds_pandemic_min': P_seeds_pandemic_min_widget,
        'P_seeds_pandemic_max': P_seeds_pandemic_max_widget,
        'num_simulations': num_simulations_widget,
        'population': population_widget,
        'infection_rate': infection_rate_widget,
        'fatality_rate': fatality_rate_widget,
        'num_facilities': num_facilities_widget,
        'num_years': num_years_widget
    }
)

### 1.1 Probability a Single Facility in a Single Year Seeds a Pandemic

The probability that a single facility seeds a pandemic in a single year is given by:

$
P_{\text{single_pandemic}} = P_{\text{release}} \times P_{\text{seeds_pandemic}}
$

Where:
- $P_{\text{release}}$ is the probability of community release from a single facility in a single year.
- $P_{\text{seeds_pandemic}}$ is the probability that a virus release seeds a pandemic. Since $P_{\text{seeds_pandemic}}$ is given as a range, we will sample from a uniform distribution between 0.05 and 0.4.

In [5]:
display(output_P_single_pandemic)

Output(outputs=({'output_type': 'display_data', 'data': {'application/vnd.plotly.v1+json': {'config': {'plotly…

### 1.2. Expected number of Pandemics

The expected number of pandemics seeded by any facility in a $num\_years$ is given by:

$
E[\text{Number of Pandemics}] = P_{\text{single_pandemic}} \times \text{num_years} \times \text{num_facilities}
$

Where:
- $\text{num_years}$ is the number of years in the simulation (typically set to 100 for a century).
- $\text{num_facilities}$ is the number of facilities in the world.
- $P_{\text{single_pandemic}}$ is the probability that a single facility seeds a pandemic in a single year.

In [6]:
display(output_P_any_facility)

Output(outputs=({'output_type': 'display_data', 'data': {'application/vnd.plotly.v1+json': {'config': {'plotly…

### 1.3. Worldwide Fatalities from the Pandemic

For each pandemic, the number of fatalities is calculated as:

$
\text{Fatalities} = E[\text{Number of Pandemics}] \times \text{Population} \times \text{Infection Rate} \times \text{Case Fatality Rate}
$

Where:
- $\text{Population}$ is the world population at the time of the pandemic.
- $\text{Infection Rate}$ is the infection rate of the pandemic.
- $\text{Case Fatality Rate}$ is the case fatality rate of the pandemic.

In [7]:
display(output_expected_fatalities)

Output(outputs=({'output_type': 'display_data', 'data': {'application/vnd.plotly.v1+json': {'config': {'plotly…